In [32]:
%run fParam.ipynb

In [33]:
%run fChannel.ipynb

In [34]:
%run fAlgorithms.ipynb

In [35]:
%run fCalculations.ipynb

In [36]:
%run fPlot.ipynb

In [37]:
import numpy as np

In [38]:
import copy

In [39]:
sys_param = myf_sys_param()

In [40]:
Num_samples = 1000
x_axis_name ="EboverNO_db_cand"

In [41]:
if(x_axis_name=="EsoverNO_db_cand"):
    
    x_axis_cand = np.arange(start=0, stop=30, step=5)
elif(x_axis_name == "EboverNO_db_cand"):
    
    x_axis_cand = np.arange(start=0, stop = 30, step=5)


In [42]:
Num_algorithms = 7

In [43]:
##constellation symbols and bits (Unit average symbol energy)
constellation = myf_constellation(sys_param)

In [44]:
Num_errors_symbol = np.zeros((Num_algorithms,np.size(x_axis_cand, axis=0)))

In [45]:
Num_errors_bits = np.zeros((Num_algorithms,np.size(x_axis_cand, axis=0)))

In [46]:
progress_per = 0

In [48]:
for ind1 in range(0,Num_samples):
    
    ##channel
    
    param_channel = {}
    
    param_channel["seed_seq"] = Num_samples + ind1
    
    channel = myf_channel(sys_param,param_channel)
    
    ##step(1): Generate a random index of the constellaion
    
    param_symbol_index = {}
    
    param_symbol_index["seed_seq"] = Num_samples + 1 + ind1
    
    symbol_index = myf_symbol_index(sys_param,param_symbol_index)
    ##print("symbol_index:"param_symbol_index)##
    ##step2:Encoding i.e: sybol_index -> constellation symbol(noteE[|x|^2]=1)
    
    x = myf_x(constellation,symbol_index)
    
#     print("x_vec: ",x_vec)

    Num_errors_symbol_temp = np.zeros((Num_algorithms,np.size(x_axis_cand,axis =0)))
    
    Num_errors_bit_temp = np.zeros((Num_algorithms,np.size(x_axis_cand,axis =0)))
    
    ###Algorithms###
    
    for ind2 in range(0,np.size(x_axis_cand,axis=0)):
        
        ##x-axis variable
        
        if(x_axis_name=="EsoverNO_dB_cand"):
            
            sys_param["EsoverNO_dB_cand"] = x_axis_cand[ind2] #Es/NO [dB]
            
            sys_param["EsoverNO"] = 10**(sys_param["EsoverNO_dB"]/10) #[no unit]
            
            sys_param["NO"] = sys_param["Es"]/sys_param["EsoverNO"] #noise energy joule
            
        elif(x_axis_name=="EboverNO_dB_cand"):
            
            sys_param["EsoverNO_dB_cand"] = x_axis_cand[ind2] + 10 * np.log10(np.log2(sys_param["constellation_size"])) #Es/NO [dB]
            
            sys_param["EsoverNO"] = 10**(sys_param["EsoverNO_dB"]/10) #[no unit]
            
            sys_param["NO"] = sys_param["Es"]/sys_param["EsoverNO"] #noise energy joule
            ##SISO####
            P_vec = myf_p(sys_param,channel,mode="SISO")
            
            ##step(3):Precoding i.e, s_vec = P_mat@x_vec(note tr(R_s_vec)=Es)
            
            s_vec = P_vec_*x
            
            #print("s_vec",s_vec)
            
            #step(4):Received symbol vector i.e y_vec = H_mat*s_vec + z_vec
            param_y_vec = {}
            param_y_vec["seed_seq"] = Num_samples + 2 + ind1
            param_y_vec["s_vec"] = s_vec
            y_vec = myf_y_vec(sys_param, channel, param_y_vec, mode="SISO")
            
            #print("y_vec:",y_vec)
            
            ##Algorithm1:SISO
            
            solutions_algorihtm_1 = myf_algorithm_1(sys_param,channel,y_vec)
            
            param_Num_errors = {}
            
            param_Num_errors["symbol_index"] = symbol_index
            
            Num_errors = myf_Num_errors(sys_param,constellation,solutions_algorithm_1,param_Num_errors)
            
            Num_errors_symbol_temp[0][ind2] = Num_errors["Num_errors_symbol"]
            
            Num_errors_bit_temp[0][ind2] = Num_errors["Num_errors_bit"]
    